In [156]:
import torch
import torch.nn as nn
# import pandas as pd
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import random
import re
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os

#from nltk.tokenize import word_tokenize
from collections import Counter

# from ...src.utils.wiki_tokens.wiki_cleaner import getMorphemeList
# import ../src/utiils/wiki_token/wiki_cleaner

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [155]:
with open("../tokens/tokens.json", "r") as file:
    tokenizer =json.load(file)

In [157]:
with open("text", "r") as file:
    wiki = file.read()

In [ ]:

# def getMorphemeList(text):
#   text = text.lower()
#   text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#   morpheme_list = word_tokenize(text)
  
#   # Count occurrences of each word
#   word_counts = Counter(morpheme_list)
  
#   # Filter out words that only appear once
#   morpheme_list = [word for word in morpheme_list if word_counts[word] > 1]
  
#   return morpheme_list


# words = getMorphemeList(wiki)

: 

In [158]:
words = wiki.split(sep=" ")
window = 2
print (words[0])
words.pop(0)

tokenized_wiki = []

for word in words:
    if word in tokenizer:
        tokenized_wiki.append(tokenizer[word])
        

In [159]:
print (tokenized_wiki[0:100])

[1247, 1959, 1, 32287, 21917, 150, 12201, 35002, 644, 10085, 36322, 5821, 32413, 21917, 32413, 27187, 1271, 32413, 27932, 7746, 21917, 32413, 27187, 36011, 32413, 32287, 17153, 30730, 35002, 15956, 1, 23224, 35832, 32827, 8643, 1554, 324, 32408, 35002, 35439, 32827, 8704, 32413, 22197, 21917, 29756, 17216, 1000, 3013, 31899, 34900, 1959, 1, 24521, 17881, 4462, 28522, 8320, 32413, 36306, 1247, 17153, 8616, 12918, 32413, 36225, 27639, 5482, 17712, 1247, 1959, 1, 24295, 23667, 17153, 32413, 3067, 32408, 1814, 34563, 1271, 29043, 2925, 1019, 32480, 1814, 21917, 35972, 32569, 1247, 1000, 32827, 26650, 29746, 32408, 531, 32413, 10412, 21917, 2330]


In [160]:
class embed_train_dataset(Dataset):
    def __init__(self, words):
        self.data = words
    
    def __len__(self):
        return len(self.data)-4
    
    def __getitem__(self, idx):
        idx = idx+2       
        sent = self.data[max(0,idx-window):min(idx+window+1,len(words))]    
        if len(sent) > 1:
            rand_idx = random.randint(0,len(sent)-1)
            target = sent[rand_idx]
            del sent[rand_idx]
            #print (sent)
            tokenized = torch.tensor(sent)

            
            return tokenized, torch.tensor(target)
        

In [135]:
dataset = embed_train_dataset(tokenized_wiki)
dataloader = DataLoader(dataset, batch_size=1,shuffle=True)

for data in dataloader:
    print (data)
    break



[17231, 12917, 22018, 17215]
[tensor([[17231, 12917, 22018, 17215]]), tensor([32412])]


In [167]:
class CBOW(torch.nn.Module):
    def __init__(self):
        super(CBOW, self).__init__()
        self.embeddings = torch.nn.Embedding(num_embeddings = len(tokenizer), embedding_dim = 200)
        self.linear = torch.nn.Linear(in_features = 200, out_features = len(tokenizer))
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = embeds.mean(dim=1)
        out = self.linear(embeds)
        #probs =  F.log_softmax(out, dim=1)
        return out

In [164]:
def train_loop():
    number_epochs = 5

    #train_wiki, val_wiki = train_test_split(words)
    os.makedirs("checkpoints", exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    dataset = embed_train_dataset(tokenized_wiki)
    dataloader = DataLoader(dataset, batch_size=128,shuffle=True)
    

    model = CBOW().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    best_loss = 100000000000000.0
    for epoch in range(number_epochs):
        model.train()
        epoch_loss = 0.0
        for X,Y in tqdm(dataloader):
            X = X.to(device)
            Y = Y.to(device)
            optimizer.zero_grad()
            pred = model(X)
            loss = F.cross_entropy(pred,Y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
        print(f"Epoch: {epoch}/{number_epochs}, loss: {epoch_loss} ")
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            save_path = f"checkpoints/storm_epoch_{epoch+1}.pt"
            torch.save(model.state_dict(), save_path)
            print(f"Model improved. Saved to: {save_path}")





In [168]:
train_loop()

  1%|          | 610/80028 [00:07<15:16, 86.62it/s] 


KeyboardInterrupt: 